In [9]:
import pymysql
import face_recognition

def insert_student(name, roll_number, image_path):
    try:
        # Connect to MySQL database
        conn = pymysql.connect(host='localhost', user='root', password='Kashak30082005', database='attendance_db')
        cursor = conn.cursor()

        # Read and encode face image
        with open(image_path, 'rb') as file:
            image_blob = file.read()  # Store image as binary

        image = face_recognition.load_image_file(image_path)
        face_encodings = face_recognition.face_encodings(image)

        if face_encodings:
            encoding_blob = face_encodings[0].tobytes()  # Convert numpy array to binary

            # Insert into database including face_image
            sql = "INSERT INTO students (name, roll, encoding, face_image) VALUES (%s, %s, %s, %s)"
            cursor.execute(sql, (name, roll_number, encoding_blob, image_blob))
            conn.commit()
            print(f"✅ Successfully inserted: {name} ({roll_number})")
        else:
            print(f"❌ No face detected in {image_path}. Skipping insertion.")

    except pymysql.MySQLError as e:
        print(f"❌ Failed to insert {name}: {e}")
    finally:
        cursor.close()
        conn.close()

# Insert student records
insert_student("KASHAK SARJU MODI", "23AIML037", "Kashak.jpg")
insert_student("BANSARI SHAILESHBHAI BHANDARI", "23AIML005", "bansari.jpg")

❌ Failed to insert KASHAK SARJU MODI: (1062, "Duplicate entry '23AIML037' for key 'students.roll'")
❌ Failed to insert BANSARI SHAILESHBHAI BHANDARI: (1062, "Duplicate entry '23AIML005' for key 'students.roll'")


In [1]:
import os
import cv2
import base64
import numpy as np
import face_recognition
import pymysql
import logging
from flask import Flask, request, jsonify, render_template
import threading
from datetime import datetime

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

DB_HOST = "localhost"
DB_USER = "root"
DB_PASSWORD = os.getenv("DB_PASSWORD", "Kashak30082005")
DB_NAME = "attendance_db"

app = Flask(__name__)

# List to store matched students (persistent during app runtime)
matched_students_list = []

def get_db_connection():
    try:
        conn = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWORD, database=DB_NAME, autocommit=True)
        return conn
    except pymysql.MySQLError as err:
        logging.error(f"Database connection error: {err}")
        return None

def load_face_encodings():
    conn = get_db_connection()
    if not conn:
        logging.error("No database connection.")
        return [], [], []

    try:
        cursor = conn.cursor()
        cursor.execute("SELECT roll, name, encoding FROM students")
        students = cursor.fetchall()

        face_encodings, student_names, student_rolls = [], [], []

        for student in students:
            try:
                encoding = np.frombuffer(student[2], dtype=np.float64)
                face_encodings.append(encoding)
                student_names.append(student[1])
                student_rolls.append(student[0])
            except Exception as e:
                logging.warning(f"Error processing student data: {student} - {e}")

        logging.info(f"Loaded {len(face_encodings)} Encodings")
        return face_encodings, student_names, student_rolls

    except pymysql.MySQLError as err:
        logging.error(f"Database error: {err}")
        return [], [], []
    finally:
        conn.close()

def mark_attendance(student_roll):
    conn = get_db_connection()
    if not conn:
        return

    try:
        cursor = conn.cursor()
        cursor.execute("INSERT INTO attendance (student_roll, timestamp) VALUES (%s, NOW())", (student_roll,))
        conn.commit()
        logging.info(f"Attendance marked for roll: {student_roll}")

    except pymysql.MySQLError as err:
        logging.error(f"Attendance marking error: {err}")

    finally:
        conn.close()

@app.route('/')
def index():
    return render_template('index.html', matched_students=matched_students_list)

@app.route('/scan_face', methods=['POST'])
def scan_face():
    global matched_students_list

    data = request.json
    image_data = data.get("image")

    if not image_data:
        return jsonify({"status": "error", "message": "No image provided"}), 400

    try:
        image_data = image_data.split(",")[1] if "," in image_data else image_data
        nparr = np.frombuffer(base64.b64decode(image_data), np.uint8)
        frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        if not face_encodings:
            return jsonify({"status": "error", "message": "No face detected"}), 400

        known_encodings, known_names, known_rolls = load_face_encodings()

        for encoding in face_encodings:
            matches = face_recognition.compare_faces(known_encodings, encoding)
            face_distances = face_recognition.face_distance(known_encodings, encoding)

            if any(matches):
                best_match_index = np.argmin(face_distances)
                student_roll = known_rolls[best_match_index]
                student_name = known_names[best_match_index]

                # Add every matched student without overriding existing data
                matched_students_list.append({
                    "roll": student_roll,
                    "name": student_name,
                    "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                })

                mark_attendance(student_roll)

        if matched_students_list:
            return jsonify({
                "status": "success",
                "matched_students": matched_students_list,
                "total_matched": len(matched_students_list)
            })

        return jsonify({"status": "error", "message": "No match found"})

    except Exception as e:
        logging.error(f"Error in scan_face: {e}")
        return jsonify({"status": "error", "message": str(e)})

def run_app():
    app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)

threading.Thread(target=run_app, daemon=True).start()


 * Serving Flask app '__main__'
 * Debug mode: on


2025-04-09 10:41:01,016 - INFO - WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.108:5000
2025-04-09 10:41:01,016 - INFO - Press CTRL+C to quit
2025-04-09 10:41:03,428 - INFO - 127.0.0.1 - - [09/Apr/2025 10:41:03] "GET / HTTP/1.1" 200 -
2025-04-09 10:41:03,571 - INFO - 127.0.0.1 - - [09/Apr/2025 10:41:03] "GET /static/script.js HTTP/1.1" 304 -
2025-04-09 10:41:03,614 - INFO - 127.0.0.1 - - [09/Apr/2025 10:41:03] "GET /favicon.ico HTTP/1.1" 404 -
2025-04-09 10:41:08,523 - INFO - Loaded 2 Encodings
2025-04-09 10:41:08,564 - INFO - Attendance marked for roll: 23AIML037
2025-04-09 10:41:08,568 - INFO - 127.0.0.1 - - [09/Apr/2025 10:41:08] "POST /scan_face HTTP/1.1" 200 -
2025-04-09 10:41:13,507 - INFO - Loaded 2 Encodings
2025-04-09 10:41:13,547 - INFO - Attendance marked for roll: 23AIML037
2025-04-09 10:41:13,547 -

In [2]:
import os
import pymysql
import pandas as pd

def export_attendance_records():
    try:
        # Ensure 'record' directory exists
        os.makedirs("record", exist_ok=True)
        
        # Connect to MySQL database
        conn = pymysql.connect(host='localhost', user='root', password='Kashak30082005', database='attendance_db')
        cursor = conn.cursor()
        
        # Fetch attendance records with student names
        query = """
        SELECT a.student_roll, s.name, a.timestamp
        FROM attendance a
        JOIN students s ON a.student_roll = s.roll
        ORDER BY a.timestamp DESC;
        """
        cursor.execute(query)
        records = cursor.fetchall()
        
        # Convert to DataFrame
        df = pd.DataFrame(records, columns=['Roll Number', 'Name', 'Timestamp'])
        
        # Save to Excel file
        file_path = "record/attendance_records.xlsx"
        df.to_excel(file_path, index=False)
        
        print(f"✅ Attendance records exported successfully to {file_path}")
        
    except pymysql.MySQLError as e:
        print(f"❌ Database error: {e}")
    except Exception as e:
        print(f"❌ Error: {e}")
    finally:
        cursor.close()
        conn.close()

if __name__ == "__main__":
    export_attendance_records()

2025-04-09 10:41:35,399 - INFO - NumExpr defaulting to 4 threads.


✅ Attendance records exported successfully to record/attendance_records.xlsx
